In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import warnings
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import platform

warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE': 2048,
    'SEED':41
}

PATH = os.getcwd() + '/data/'
if platform.system() == 'Darwin':
    LOADPATH = '/Users/a1r/Desktop/DL/timeseries_new_data/'
else:
    LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

## CNN-LSTM MODEL
![스크린샷 2023-08-04 오후 5 16 00](https://github.com/Megvii-BaseDetection/YOLOX/assets/103639510/6e612140-39b5-4198-bb14-81ff2aca5f22)


In [ ]:
class Conv1d_LSTM(nn.Module):
    def __init__(self, in_channel=9, hidden_size = 256, out_channel=CFG['PREDICT_SIZE']):
        super(Conv1d_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.conv1d_1 = nn.Conv1d(in_channels=in_channel,
                                out_channels=16,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        
        self.conv1d_2 = nn.Conv1d(in_channels=16,
                                out_channels=32,
                                kernel_size=3,
                                stride=1,
                                padding=1)
        
        self.lstm = nn.LSTM(input_size = 32,
                            hidden_size=hidden_size,
                            num_layers=1,
                            bias=True,
                            bidirectional=False,
                            batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(hidden_size//2, out_channel)
        )

        self.actv = nn.ReLU()
        # self.dense1 = nn.Linear(hidden_size, hidden_size//2)
        # self.dropout = nn.Dropout()
        # self.dense2 = nn.Linear(hidden_size//2, out_channel)
    
    def init_hidden(self, batch_size, device):
        return (
            torch.zeros(1, batch_size, self.hidden_size, device = device),  # h0
            torch.zeros(1, batch_size, self.hidden_size, device = device)   # c0
        )

    def forward(self, x):
	# Raw x shape : (B, TRAIN_WINDOW_SIZE, in_channel) => (B, 90, 9)
        
        # Shape : (B, F: in_channel, S: TRAIN_WINDOW_SIZE) => (B, 9, 90)
        x = x.transpose(1, 2)
        # Shape with Conv1d_1 : (B, F, S) == (B, C, S) || C = channel => (B, 16, 90)
        x = self.conv1d_1(x)
        ## Shape with Conv1d_2 : (B, C, S) => (B, 32, 90)
        x = self.conv1d_2(x)
        ## Reshape : (B, S, C) == (B, S, F) => (B, 90, 32)
        x = x.transpose(1, 2)
        
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM Layer
        lstm_out, hidden = self.lstm(x, hidden)

        # Only use the last output(-1) sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))
        x = output.squeeze(1)
        
        # # Shape : (B, 512)
        # x = self.dense1(x)
        # # ReLU
        # x = F.relu(x)
        # # Shape : (B, H) => (B, 256)
        # x = self.dropout(x)
        # # Shape : (B, O) // O = output => (B, 21)
        # x = self.dense2(x)

        return x

## Loss

#### Metric

In [ ]:
def pseudo_sfa(pred, df):
    pred_length = pred.shape[1] - 1
    true = df.iloc[:, -pred_length:].reset_index() \
        .rename(columns={"index": "ID"})

    main_id = {}
    for main_cat in df["대분류"].unique():
        main_id[main_cat] = df.query("대분류==@main_cat")["ID"].to_list()

    psfa = []
    for main_cat in main_id.keys():
        indices = true["ID"].isin(main_id[main_cat])

        true_arr = true[indices].iloc[:, 1:].to_numpy()
        pred_arr = pred[indices].iloc[:, 1:].to_numpy()

        eps = np.ones((true_arr.shape)) / 1e8

        true_sum = true_arr.sum(axis=0)
        true_sum = np.stack([true_sum]*len(true_arr)) + eps
        true_rate = true_arr / true_sum

        abs_error = np.abs(true_arr - pred_arr)
        denom = np.maximum(true_arr, pred_arr+eps)
        
        score = 1 - (1 / true_arr.shape[1]
                     * (abs_error / denom) * true_rate).sum()
        psfa.append(score)
        print(main_cat, score)

    return np.mean(psfa)

#### Loss Function

In [ ]:
class PsfaLoss(nn.Module):
    def __init__(self, scaler, df):
        super().__init__()
        self.scaler = scaler
        self.main_cats = df.groupby("대분류")["ID"].unique().values
    
    def forward(self, pred, true):
        # pred: [batch_size, length, products(15890)]
        pred = pred * torch.tensor(self.scaler.denom) \
            + torch.tensor(self.scaler.min_val)
        true = true * torch.tensor(self.scaler.denom) \
            + torch.tensor(self.scaler.min_val)

        L1scaled = torch.abs(true-pred) / torch.maximum(pred, true+1e-8)
        
        rate = torch.zeros_like(true)
        for i in range(len(self.main_cats)):
            rate[:, :, self.main_cats[i]] = \
                true[:, :, self.main_cats[i]] \
                / (true[:, :, self.main_cats[i]].sum(dim=-1, keepdim=True) + 1e-8) \
                / len(self.main_cats)
        return (L1scaled * rate).sum() / (true.shape[0] * true.shape[1])

## Training Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

#### Call the Dataset Array

In [ ]:
train_input = np.load(LOADPATH + 'train_input_minmax.npy')
train_target = np.load(LOADPATH + 'train_target_minmax.npy')

In [ ]:
print(train_input.shape, train_target.shape)
print(train_input[:2])

In [ ]:
data_len = len(train_input)
train_dataset = CustomDataset(train_input[:-int(data_len*0.2)], train_target[:-int(data_len*0.2)])
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(train_input[-int(data_len*0.2):], train_target[-int(data_len*0.2):])
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    train_loss = {}
    val_loss = {}
    
    # Epoch
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss_li = []
        train_mae = []
    
    # Iteration
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss_li.append(loss.item())
        
        val_loss_li = validation(model, val_loader, criterion, device)
        val_loss_mean = np.mean(val_loss_li)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss_li):.5f}] Val Loss : [{val_loss_mean:.5f}]')
        
        if best_loss > val_loss_mean:
            best_loss = val_loss_mean
            best_model = model
            print('Model Saved')

        train_loss[epoch] = train_loss_li
        val_loss[epoch] = val_loss_li

    return best_model, train_loss, val_loss

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return val_loss

In [ ]:
model = Conv1d_LSTM()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG['LEARNING_RATE'])
infer_model, train_loss, val_loss = train(model, optimizer, train_loader, val_loader, device)

In [ ]:
SAVE_PATH = os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth'
torch.save(infer_model.state_dict(), SAVE_PATH)

In [ ]:
train_loss = np.array(train_loss)
val_loss = np.array(val_loss)

# loss값 저장
np.save(PATH + 'train_loss_drop_0.4', train_loss)
np.save(PATH + 'val_loss_drop_0.4', val_loss)

In [6]:
test_input = np.load(LOADPATH + 'test_input_minmax.npy')
min_ = np.load(PATH + 'MIN.npy')
max_ = np.load(PATH + 'MAX.npy')

In [7]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [8]:
model = Conv1d_LSTM()
if platform.system() == "Darwin":
    model.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth', map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(os.getcwd() + '/cnn_lstm_drop_0.4_1e_4.pth'))

### Submit Sales' Predictions

In [10]:
prediction = inference(model, test_loader, device)
prediction.shape

  0%|          | 0/8 [00:00<?, ?it/s]

(15890, 21)

In [ ]:
prediction[0]

In [11]:
def return_scaler(arr, min_ = min_, max_ = max_):
    denom = max_ - min_
    for i in tqdm(range(len(arr))):
        arr[i] = (arr[i]*denom[i]) + min_[i]
    
    return arr

prediction_unscale = np.around(return_scaler(prediction))
prediction_unscale

  0%|          | 0/15890 [00:00<?, ?it/s]

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
submit = pd.read_csv(PATH + '/sample_submission.csv') # submit은 15690개 상품 / 예측은 15682개 상품
submit.iloc[:, 1:] = prediction_unscale
submit

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,0,0,0,0,0,0,1,0,1,...,0,0,1,1,1,1,1,1,1,1
3,3,0,0,1,1,1,1,1,2,1,...,2,2,2,2,2,2,2,2,2,2
4,4,1,2,2,2,2,2,2,2,2,...,2,2,3,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,0,2,3,3,3,3,3,4,4,...,4,4,4,4,5,5,5,5,5,5
15886,15886,0,2,2,2,3,3,3,3,3,...,3,3,3,4,4,4,4,4,4,4
15887,15887,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15888,15888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
SAVE_SUBMIT = os.getcwd() + '/data/cnn_lstm_drop(0.4).csv'
submit.to_csv(SAVE_SUBMIT, index_label=False)